In [1]:
def iee_b3():
    '''Funca para importar dados do site da B3 sobre a ibov, os dados no fim serão organizados em um dataframe para podermos usá-los futuramente'''
    # Importando bibliotecas
    from selenium import webdriver # Usado para importar os dados da url
    import time # usado para gerar um tempo de espera enquanto se utiliza o selenium
    from bs4 import BeautifulSoup # manipular o html
    import pandas as pd # criar a DF que será trabalhada
    
    # definindo a url da b3
    url = "https://sistemaswebb3-listados.b3.com.br/indexPage/day/IEEX?language=pt-br"
    
    # iniciando o webdriver, que deve estar com o driver na pasta do arquivo para funcionar, nesse caso será utilizado do Chrome
    browser = webdriver.Chrome()
    
    # Entrando na url informada
    browser.get(url)
    browser.find_element_by_id('segment').send_keys('Setor de Atuação')
    time.sleep(.5)
    html = browser.page_source

    browser.close() 
    
    # Usando o beatiful soup e convertendo o arquivo para podermos selecionar os dados que precisamos
    bs = BeautifulSoup(html, 'html.parser')
    
    # Criando as listas onde iremos armazenar os dados do html
    setor, cod, acao, tipo, qtde_t, part, part_a = [], [], [], [], [], [], []
    
    # Usando a função find_all para selecionarmos apenas 'td' que contém os dados que usaremos e armazenando eles na variável linhas
    linhas = bs.find_all('td')
    
    # Criando uma variável que será usada no while loop abaixo
    i = 0
    
    # While loop para podermos armazenar os dados em suas respectivas colunas, onde:
    # temos 7 colunas então pegamos o primeiro elemento 0 e somando 1 até chegarmos ao último elemento para a coluna [i+6] = ao setimo elemento
    # por fim somando 7 para pularmos para a próxima linhas e seguir a lógica
    # o loop só irá para quando o número de linhas for maior do que o que os dados apresentados, para isso usados o lógica len(linhas) que nos entrega a quantia de dados
    # e como i é somado de 7 a cada iteração o loop só irá para quando todos os dados forem indexidaodos nas suas devidas colunas
    while i < (len(linhas)):
        setor.append(linhas[i].text)
        cod.append(linhas[i+1].text)
        acao.append(linhas[i+2].text)
        tipo.append(linhas[i+3].text)
        qtde_t.append(linhas[i+4].text)
        part.append(linhas[i+5].text)
        part_a.append(linhas[i+6].text)
        i += 7
        
    # Criando um df para armazearmos os dados    
    df = pd.DataFrame({'Setor': setor[:-1],
                  'Acao': cod[:-1],
                  'Empresa': acao[:-1],
                  'Qntd_teorica': qtde_t[:-1],
                  'Part_%': part[:-1]}) # filtramos para eliminar as últimas linhas, pois a b3 entrega uma soma ao final que não queremos
    # Separando os setores e subsetores em colunas
    df['SubSetor'] = df['Setor'].apply(lambda s: s[s.rfind('/')+1:].strip())
    df['Setor'] = df['Setor'].apply(lambda s: s[:s.rfind('/')])
    
    # Convertendo os valores de string para int
    df['Qntd_teorica'] = df['Qntd_teorica'].apply(lambda s: s.replace(".", ""))
    df['Qntd_teorica'] = pd.to_numeric(df['Qntd_teorica'])
    df['Part_%'] = df['Part_%'].apply(lambda s: s.replace(",", ""))
    df['Part_%'] = pd.to_numeric(df['Part_%'])/1000
    df.sort_values('Part_%', ascending = False, inplace = True)
    
    # Criando uma colunas com a % acumulada
    df['Part_%_acum'] = df['Part_%'].cumsum()
    
    # Resetando o index
    df.reset_index(drop = True, inplace = True)
    
    # fim
    return df

In [7]:
def import_indicadores(acao):   
    from selenium import webdriver # Usado para importar os dados da url
    import time # usado para gerar um tempo de espera enquanto se utiliza o selenium
    from bs4 import BeautifulSoup # manipular o html
    import pandas as pd # criar a DF que será trabalhada
    from datetime import date

    url = f"https://statusinvest.com.br/acoes/{acao.lower()}"

    browser = webdriver.Chrome()

    # Entrando na url informada
    browser.get(url)
    time.sleep(2)
    element = browser.find_elements_by_xpath('/html/body/main/div[2]/div/div[6]/div[1]/div/button[2]/div/span')
    for i in element: i.click()
    time.sleep(.5)    
    element = browser.find_elements_by_xpath('/html/body/main/div[2]/div/div[6]/div[3]/div[1]/div[2]/div[2]/ul/li[2]/a')
    for i in element: i.click()
    time.sleep(.5)
    html = browser.page_source

    browser.close() 

    # Usando o beatiful soup e convertendo o arquivo para podermos selecionar os dados que precisamos
    bs = BeautifulSoup(html, 'html.parser')

    colunas = bs.find_all('div', class_='th w-100')
    linhas = bs.find_all('div', class_='td w-100')

    a = []
    for coluna in colunas:  a.append(coluna.text)

    b = []
    for i in linhas: b.append(i.text)   

    while True:
        i = 0
        intervalos = []
        while i < len(linhas):
            try:
                if a.index('ATUAL', i) == i:
                    intervalos.append(i)
            except ValueError:
                break
            i += 1
        break

    while True:
        i = 0
        x = []
        y = []
        try:
            while i < len(intervalos):
                x.append(intervalos[i]), y.append(intervalos[i+1])
                i += 1
            break
        except IndexError:
            y.append(len(linhas))
        break
        
    a_1 = {}
    for i in range(1, len(x)+1): a_1[f'{i}'] = a[x[i-1]:y[i-1]]

    a_2 = {}
    for i in range(1, len(x)+1): a_2[f'{i}'] = len(a_1[f'{i}'])

    obj = {}
    for i in range(1, len(max(a_1.values()))+1): obj[max(a_1.values())[i-1]] = []


    while True:
        _a = 0
        _b = 0
        _1 = (len(a_1['1']))*14
        _2 = (len(a_1['2']))*6 + _1
        _3 = (len(a_1['3']))*4 + _2
        _4 = (len(a_1['4']))*4 + _3
        _5 = (len(a_1['5']))*2 + _4

        while _a < len(obj):
            obj[f'{max(a_1.values())[_a]}'].append(b[_b: _1 :(len(a_1['1']))] if _a+1 <= len(a_1['1']) else [0*i for i in range(14)])
            _a += 1
            _b += 1

        _a = 0
        _b = _1     
        while _a < len(obj):
            obj[f'{max(a_1.values())[_a]}'].append(b[_b: _2 :(len(a_1['2']))] if _a+1 <= len(a_1['2']) else [0*i for i in range(14)])
            _a += 1
            _b += 1

        _a = 0
        _b = _2     
        while _a < len(obj):
            obj[f'{max(a_1.values())[_a]}'].append(b[_b: _3 :(len(a_1['3']))] if _a+1 <= len(a_1['3']) else [0*i for i in range(14)])
            _a += 1
            _b += 1    

        _a = 0
        _b = _3     
        while _a < len(obj):
            obj[f'{max(a_1.values())[_a]}'].append(b[_b: _4 :(len(a_1['4']))] if _a+1 <= len(a_1['4']) else [0*i for i in range(len(obj)+1)])
            _a += 1
            _b += 1  

        _a = 0
        _b = _4     
        while _a < len(obj):
            obj[f'{max(a_1.values())[_a]}'].append(b[_b: _5 :(len(a_1['5']))] if _a+1 <= len(a_1['5']) else [0*i for i in range(14)])
            _a += 1
            _b += 1      
        break 

    indicadores = ['DY', 'PL','PEG_RATIO', 'P_PV', 'EV_EBITIDA', 'EV_EBIT', 'P_EBITIDA', 'P_EBIT',
          'VPA', 'P_ATIVO', 'LPA', 'S_SR', 'P_CAP_GIRO', 'P_ATIVO_CIRC_LIQ', 'DIV_LIQUIDA_PL',
          'DIV_LIQUIDA_EBITIDA', 'DIV_LIQUIDA_EBIT', 'PL_ATIVOS', 'PASSIVOS_ATIVOS', 'LIQ_CORRENTE',
          'M_BRUTA', 'M_EBITIDA', 'M_EBIT', 'M_LIQUIDA', 'ROE', 'ROA', 'ROIC', 'GIRO_ATIVOS',
          'CAGR_RECEITAS_5_ANOS', 'CAGR_LUCROS_5_ANOS']
    ind = {'indicadores':indicadores}
    ind.update({ f'{i}':obj[f'{i}'][0] + obj[f'{i}'][1] + obj[f'{i}'][2] + obj[f'{i}'][3] + obj[f'{i}'][4] for i in max(a_1.values())})
    df = pd.DataFrame(ind) 
    return df

In [3]:
iee_b3 = iee_b3()

In [4]:
iee_b3

,Setor,Acao,Empresa,Qntd_teorica,Part_%,SubSetor,Part_%_acum
0,Utilidade Públ,CMIG4,CEMIG,13600,6.198,Energ Elétrica,6.198
1,Utilidade Públ,ELET3,ELETROBRAS,4900,5.910,Energ Elétrica,12.108
2,Utilidade Públ,ENGI11,ENERGISA,4200,5.854,Energ Elétrica,17.962
3,Utilidade Públ,ENBR3,ENERGIAS BR,10100,5.787,Energ Elétrica,23.749
4,Utilidade Públ,CESP6,CESP,7800,5.748,Energ Elétrica,29.497
5,Utilidade Públ,TRPL4,TRAN PAULIST,7600,5.712,Energ Elétrica,35.209
6,Utilidade Públ,EGIE3,ENGIE BRASIL,4900,5.699,Energ Elétrica,40.908
7,Utilidade Públ,TAEE11,TAESA,5000,5.666,Energ Elétrica,46.574
8,Utilidade Públ,ALUP11,ALUPAR,7500,5.659,Energ Elétrica,52.233
9,Utilidade Públ,CPLE6,COPEL,28800,5.654,Energ Elétrica,57.887


In [8]:
import time
for i in iee_b3.Acao:
    import_indicadores(i)
    print('{} -> ok'.format(i))

CMIG4 -> ok
ELET3 -> ok
ENGI11 -> ok
ENBR3 -> ok
CESP6 -> ok
TRPL4 -> ok
EGIE3 -> ok
TAEE11 -> ok
ALUP11 -> ok
CPLE6 -> ok
COCE5 -> ok
CPFE3 -> ok
EQTL3 -> ok
OMGE3 -> ok
ENEV3 -> ok
NEOE3 -> ok
LIGT3 -> ok
AESB3 -> ok
